In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
!ls

adc.json  drive  sample_data


In [2]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
hsi = []
for i in range(3):
  img_path = "pca/hsi_pc" + str(i) + ".png"
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  hsi.append(img)
        
hsi = np.array(hsi)

lidar = cv2.imread("pca/lidar.png", cv2.IMREAD_GRAYSCALE)
lidar = lidar.reshape(1, lidar.shape[0], lidar.shape[1])

In [5]:
print(hsi.shape)
print(lidar.shape)

(3, 349, 1905)
(1, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]

In [0]:
hsi_train_data = []
hsi_test_data = []
lidar_train_data = []
lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            hsi_train_data.append(hsi[:, i, j])
            lidar_train_data.append(lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            hsi_test_data.append(hsi[:, i, j])
            lidar_test_data.append(lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
hsi_train_data = np.array(hsi_train_data)
lidar_train_data = np.array(lidar_train_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [10]:
print(hsi_train_data.shape)
print(lidar_train_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 3)
(2832, 1)
(2832,)
(12197,)


In [11]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [0]:
HSI_PATCH_SIZE = 27
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005

In [13]:
padded_hsi = np.lib.pad(hsi, ((0,0), (HSI_PATCH_SIZE//2, HSI_PATCH_SIZE//2), (HSI_PATCH_SIZE//2,HSI_PATCH_SIZE//2)), 'reflect')
padded_lidar = np.lib.pad(lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')
print(padded_hsi.shape)
print(padded_lidar.shape)

(3, 375, 1931)
(1, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
hsi_train_patches = []
hsi_test_patches = []
lidar_train_patches = []
lidar_test_patches = []
for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      hsi_train_patches.append(get_patches(padded_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      lidar_train_patches.append(get_patches(padded_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      hsi_test_patches.append(get_patches(padded_hsi, HSI_PATCH_SIZE, i+HSI_PATCH_SIZE//2, j+HSI_PATCH_SIZE//2))
      lidar_test_patches.append(get_patches(padded_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [16]:
hsi_train_patches = np.array(hsi_train_patches)
hsi_test_patches = np.array(hsi_test_patches)
lidar_train_patches = np.array(lidar_train_patches)
lidar_test_patches = np.array(lidar_test_patches)
print(hsi_train_patches.shape)
print(hsi_test_patches.shape)
print(lidar_train_patches.shape)
print(lidar_test_patches.shape)

(2832, 27, 27, 3)
(12197, 27, 27, 3)
(2832, 41, 41, 1)
(12197, 41, 41, 1)


In [17]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = hsi_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

hsi_model = Sequential()
hsi_model.add(InputLayer(input_shape=(HSI_PATCH_SIZE, HSI_PATCH_SIZE, BANDS)))
hsi_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
hsi_model.add(BatchNormalization())
hsi_model.add(MaxPooling2D(pool_size=2, strides=2))
hsi_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
hsi_model.add(BatchNormalization())
hsi_model.add(MaxPooling2D(pool_size=2, strides=2))


hsi_model.add(Flatten())
hsi_model.add(Dense(FC1, activation='relu'))
hsi_model.add(Dropout(0.75))
hsi_model.add(Dense(FC2, activation='relu'))
hsi_model.add(Dropout(0.6))
hsi_model.add(Dense(NUM_CLS, activation='softmax'))


hsi_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 14, 14, 500)       54500     
_________________________________________________________________
batch_normalization (BatchNo (None, 14, 14, 500)       2000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 500)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 4, 4, 100)         1250100   
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 100)         400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
__________

In [18]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

lidar_model = Sequential()
lidar_model.add(InputLayer(input_shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS)))
lidar_model.add(Conv2D(kernel_size=6, strides=2, filters=75, padding='same', activation='relu', name='conv1'))
lidar_model.add(BatchNormalization())
lidar_model.add(MaxPooling2D(pool_size=2, strides=2))
lidar_model.add(Conv2D(kernel_size=5, strides=2, filters=100, padding='same', activation='relu', name='conv2'))
lidar_model.add(BatchNormalization())
lidar_model.add(MaxPooling2D(pool_size=2, strides=2))


lidar_model.add(Flatten())
lidar_model.add(Dense(FC1, activation='relu'))
lidar_model.add(Dropout(0.75))
lidar_model.add(Dense(FC2, activation='relu'))
lidar_model.add(Dropout(0.6))
lidar_model.add(Dense(NUM_CLS, activation='softmax'))


lidar_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 21, 21, 75)        2775      
_________________________________________________________________
batch_normalization_2 (Batch (None, 21, 21, 75)        300       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 75)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 5, 5, 100)         187600    
_________________________________________________________________
batch_normalization_3 (Batch (None, 5, 5, 100)         400       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
__________

In [19]:
hsi_model.load_weights('Models/hsi_model_weights.h5')
lidar_model.load_weights('Models/lidar_model_weights.h5')

ValueError: ignored

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [21]:
test_cls = test_labels - 1
prediction = hsi_model.predict(hsi_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.6285152086578667
Average Accuracy of training samples :  0.6665988960020399
Kappa statistics of training samples :  0.5958508339430351
Each class accuracy of training samples :  [0.8247639  0.66791745 0.5907781  0.91914191 0.74119339 0.93442623
 0.75595238 0.5035461  0.75259679 0.62673392 0.35395932 0.50643087
 0.54761905 0.62457338 0.64935065]
Confusion matrix : [[ 786   95    0   63    0    0    0    0    0    0    3    0    0    0
     6]
 [  64  712    0  209    0    0    4    0    1    0    0    0    0    5
    71]
 [   2    0  205   33    0    5    7   11    0   21    1   18    1    2
    41]
 [   3   46    0  557    0    0    0    0    0    0    0    0    0    0
     0]
 [   0    0    0    3 1031    0    0   49   47    1    0  176    0   56
    28]
 [   2    0    0    0    0  114    0    0    1    0    0    0    0    0
     5]
 [  11   39    3   73    0   19  889   21   64    0   36    0   21    0
     0]
 [   0    1    3   19    4    0 

In [22]:
test_cls = test_labels - 1
prediction = lidar_model.predict(lidar_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :", confusion_matrix)

Overall Accuracy of training sapmles :  0.026399934410100846
Average Accuracy of training samples :  0.009662371033273704
Kappa statistics of training samples :  -0.0017591517152106828
Each class accuracy of training samples :  [0.         0.         0.         0.         0.08706953 0.00504789
 0.         0.         0.         0.         0.         0.
 0.         0.         0.05281815]
Confusion matrix : [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [126 116  19 104 134  79  97  43 195 149  98 139  51 122  67]
 [544 556 185 789 638  39 862 639 785 575 823 705 204 125 257]
 [ 12   0   0   2   7   0   0   0   1   1   0   0   0   0   0]
 [ 30   3   0   0   4  21   0   0   2   0   0   7   0   0   0]
 [ 18   0   0   0   0   3   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [0]:
from tensorflow.python.keras.models import Model
intermediate_layer_hsi_model = Model(hsi_model.layers[0].input,outputs=hsi_model.layers[6].output)
intermediate_layer_lidar_model = Model(lidar_model.layers[0].input,outputs=lidar_model.layers[6].output)

hsi_train_flatten = intermediate_layer_hsi_model.predict(hsi_train_patches)
lidar_train_flatten = intermediate_layer_lidar_model.predict(lidar_train_patches)

hsi_test_flatten = intermediate_layer_hsi_model.predict(hsi_test_patches)
lidar_test_flatten = intermediate_layer_lidar_model.predict(lidar_test_patches)

In [24]:
print(hsi_train_flatten.shape)
print(lidar_train_flatten.shape)
print(hsi_test_flatten.shape)
print(lidar_test_flatten.shape)

(2832, 400)
(2832, 400)
(12197, 400)
(12197, 400)


In [0]:
def get_outer_products(v1,v2):
  products = []
  for i in range(v1.shape[0]):
    products.append(np.outer(v1[i], v2[i]))
  return np.array(products)

In [0]:
train_outer_products = get_outer_products(hsi_train_flatten, lidar_train_flatten)
#test_outer_products = get_outer_products(hsi_test_flatten, lidar_test_flatten)

In [27]:
train_outer_products = train_outer_products.reshape(train_outer_products.shape[0], train_outer_products.shape[1], train_outer_products.shape[2], 1)
print(train_outer_products.shape)
#print(test_outer_products.shape)

(2832, 400, 400, 1)


In [31]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = train_outer_products.shape[3]
NUM_CLS = train_one_hot.shape[1]
BATCH_SIZE = 25

fusion_model = Sequential()
fusion_model.add(InputLayer(input_shape=(train_outer_products.shape[1], train_outer_products.shape[2], BANDS)))
fusion_model.add(Conv2D(kernel_size=12, strides=8, filters=256, padding='same', activation='relu', name='conv1'))
fusion_model.add(BatchNormalization())
fusion_model.add(MaxPooling2D(pool_size=2, strides=3))
fusion_model.add(Conv2D(kernel_size=12, strides=8, filters=128, padding='same', activation='relu', name='conv2'))
fusion_model.add(BatchNormalization())
fusion_model.add(MaxPooling2D(pool_size=2, strides=2))


fusion_model.add(Flatten())
fusion_model.add(Dense(FC1, activation='relu'))
fusion_model.add(Dropout(0.75))
fusion_model.add(Dense(FC2, activation='relu'))
fusion_model.add(Dropout(0.5))
fusion_model.add(Dense(NUM_CLS, activation='softmax'))


fusion_model.summary()

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
fusion_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 50, 50, 256)       37120     
_________________________________________________________________
batch_normalization_10 (Batc (None, 50, 50, 256)       1024      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 17, 17, 256)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 3, 3, 128)         4718720   
_________________________________________________________________
batch_normalization_11 (Batc (None, 3, 3, 128)         512       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 128)               0         
__________

In [34]:
history = fusion_model.fit(train_outer_products, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.2634 - acc: 0.9103
Epoch 2/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.2193 - acc: 0.9258
Epoch 3/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.1962 - acc: 0.9379
Epoch 4/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.1886 - acc: 0.9428
Epoch 5/50
2832/2832 [==============================] - 7s 3ms/step - loss: 0.2059 - acc: 0.9311
Epoch 6/50
2832/2832 [==============================] - 8s 3ms/step - loss: 0.1834 - acc: 0.9389
Epoch 7/50
2832/2832 [==============================] - 8s 3ms/step - loss: 0.1601 - acc: 0.9460
Epoch 8/50
2832/2832 [==============================] - 8s 3ms/step - loss: 0.1458 - acc: 0.9513
Epoch 9/50
2832/2832 [==============================] - 8s 3ms/step - loss: 0.1325 - acc: 0.9559
Epoch 10/50
2832/2832 [==============================] - 8s 3ms/step - loss: 0.1277 - acc: 0.9594
Epoch 11/50
2832/2832 [======

In [35]:
test_cls = test_labels - 1

prediction = []



for i in range(12197):
  prediction.append(fusion_model.predict(np.outer(hsi_test_flatten[i], lidar_test_flatten[i]).reshape(1, 400, 400, 1)).argmax(axis=-1))


prediction = np.array(prediction)


from sklearn import metrics, preprocessing

overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :\n", confusion_matrix)

Overall Accuracy of training sapmles :  0.6254816758219234
Average Accuracy of training samples :  0.6504374378023814
Kappa statistics of training samples :  0.5926143317735451
Each class accuracy of training samples :  [0.86622074 0.6580534  0.56907895 0.92093023 0.77365632 0.74342105
 0.75578406 0.45765766 0.74768089 0.72357724 0.33111481 0.52276708
 0.45411765 0.78645833 0.44604317]
Confusion matrix :
 [[ 777   49    0   67    0    0    0    0    2    0    2    0    0    0
     0]
 [  64  764    0  191    0    0    9    6    0    0    1    0    0   39
    87]
 [  40    0  173   19    0    0   10    8    2    0    1    1    0    1
    49]
 [   2   37    0  594    0    2    4    0    0    0    2    0    0    0
     4]
 [   0    0    0    3 1022    0    0   42   36    0    4  123    0   49
    42]
 [   5    0   15    1    0  113    0    6    0    0    0    0    0    0
    12]
 [   9   38    0   80    0   22  882   20   73    0   33    0   10    0
     0]
 [   0    8    2   23    9    0

In [0]:
fusion_model.save_weights('Models/outer_fusion_model_weights.h5')